In [1]:
import pandas as pd

In [2]:
df_sent_train = pd.read_csv("Data/training.1600000.processed.noemoticon.csv").iloc[::-1]

In [3]:
df_sent_train.head()

,Sentiment,ID,Date,QUERY,Username,OriginalTweet
1048574,4,1960186607,Fri May 29 07:33:45 PDT 2009,NO_QUERY,sugababez,cant wait til her date this weekend
1048573,4,1960186445,Fri May 29 07:33:44 PDT 2009,NO_QUERY,jonasobsessedx,@DestinyHope92 im great thaanks wbuu?
1048572,4,1960186429,Fri May 29 07:33:44 PDT 2009,NO_QUERY,Falchion,@ShaDeLa same here say it like from the Termi...
1048571,4,1960186409,Fri May 29 07:33:43 PDT 2009,NO_QUERY,OffRoad_Dude,Mid-morning snack time... A bowl of cheese noo...
1048570,4,1960186342,Fri May 29 07:33:44 PDT 2009,NO_QUERY,Madelinedugganx,My GrandMa is making Dinenr with my Mum


In [4]:
import string
import spacy
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load('en_core_web_sm')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)
    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lower_ for word in mytokens ]
    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]
    # return preprocessed list of tokens
    return mytokens
print(spacy_tokenizer(df_sent_train['OriginalTweet'][1]))

# Custom transformer using spaCy
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

# Basic function to clean the text
def clean_text(text):
    # Removing spaces and converting text into lowercase
    return text.strip().lower()

['@kenichan', 'dived', 'times', 'ball', 'managed', 'save', '50', ' ', 'rest', 'bounds']


In [5]:
import random
from sklearn.model_selection import train_test_split

OT = df_sent_train['OriginalTweet'] # the features we want to analyze
pre_sent = df_sent_train['Sentiment'] # the labels, or answers, we want to test against

Sent = []
neg = 0
pos = 0
for rank in pre_sent:    
    if rank == 0:
        neg = neg +1
        if(neg <= pos*1.25):
            Sent.append(0)
    else: 
        Sent.append(1)
        pos = pos +1

print(pos)
print(neg)
c = list(zip(OT,Sent))
random.shuffle(c)
OT, Sent = zip(*c)
    
OT_train, OT_val, Sent_train, Sent_val = train_test_split(OT, Sent, test_size=0.15)
print(len(OT_train))
print(len(Sent_train))

248576
799999
475401
475401


In [6]:
#Precision, Recall and F1 are all based on if the tweet are classified 
#as "Not Negative" in any way
from sklearn import metrics
def report(method, predictions, truth):
    print(method)
    print("Accuracy:",metrics.accuracy_score(truth, predictions))
    print("Precision:",metrics.precision_score(truth, predictions))
    print("Recall:",metrics.recall_score(truth, predictions))
    F1 = 2*((metrics.precision_score(truth, predictions)*metrics.recall_score(truth, predictions))
            /(metrics.precision_score(truth, predictions)+metrics.recall_score(truth, predictions)))
    print("F1: ", F1)
    return [method, metrics.accuracy_score(truth, predictions), metrics.precision_score(truth, predictions), 
            metrics.recall_score(truth, predictions), F1]

In [ ]:
# your code here
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(bootstrap = False, 
                            criterion = 'entropy',
                            n_estimators = 100, )
forestpipe = Pipeline([('bow',CountVectorizer()),
                     ('clf',clf)])
forestpipe.fit(OT_train[:20000],Sent_train[:20000])
predicted = forestpipe.predict(OT_val)
report("Random Forest Classifier", predicted, Sent_val)

Random Forest Classifier
Accuracy: 0.7613683771380892
Precision: 0.7670815183571873
Recall: 0.6629557921910294


In [ ]:
s1 = pd.read_csv("Data/Transcripts/MFs1CSV.csv")
s2 = pd.read_csv("Data/Transcripts/MFs2CSV.csv")
s3 = pd.read_csv("Data/Transcripts/MFs3CSV.csv")
s5 = pd.read_csv("Data/Transcripts/MFs5CSV.csv")
seasons = [s1, s2, s3, s5]

In [ ]:
s1

In [ ]:
phil = []
claire = []
haley = []
alex = []
luke = []
gloria = []
jay = []
manny = []
cam = []
mitch = []
lily = []
chars = [phil, claire, haley, alex, luke, gloria, jay, manny, cam, mitch, lily]

In [ ]:
#Returns an array structured such as 
#array[Character][Season]

def Char_Sent(Characters, Data):
    row = 0
    full_p = []
    for char in Characters:
        predictions = []
        for season in Data:
            predict = forestpipe.predict(season.iloc[row].dropna())
            predictions.append(predict)
        full_p.append(predictions)
        row = row+1
    return full_p

In [ ]:
char_sentiments = Char_Sent(chars, seasons)

In [ ]:
#Returns an array structued as 
#array[Season][Character]
def Season_Sent(Data):
    seasonal = []
    for season in Data:
        predictions = []
        for x in range(11):
            predict = forestpipe.predict(season.iloc[x][1:].dropna())
            predictions.append(predict)
        seasonal.append(predictions)
    return seasonal

In [ ]:
season_sentiments = Season_Sent(seasons)

In [ ]:
char_sentiments

In [ ]:
season_sentiments